In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import glm
import pandas as pd
import pingouin as pg
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Proximate/Distance Angle Analysis Between Focal and Target Mice

In [ ]:
# read in files like this. 
mouse_id = ['Animal#1', 'Animal#2', 'Animal#3', 'Animal#4', 'Animal#5']
data_dir = '/Users/File Path/'
suffix = 'your suffix.csv'
data_files = [data_dir + mouse + suffix for mouse in mouse_id]
print(data_files)

In [ ]:
# read in to pandas like this. 
df_list = [pd.read_csv(file) for file in data_files]

# add a column with the mouse id
for i, id in enumerate(mouse_id):
    df_list[i] = df_list[i].assign(mouse_id=id)
    
# merge datasets.
df_merge = pd.concat(df_list, axis=0, ignore_index=True)

In [ ]:
#get rid of all the columns we're not using for this analysis
df_merge = df_merge.drop (['time','NN_sniff','NN_new','dFF'], axis=1)
df_merge

In [ ]:
#Bin into central, peripheral, and facing away
#Then bin by prox/distance between focal and target mice

df_merge.loc[df_merge['angle'].between(0, 40, 'both'), 'bin_angle'] = '40'
df_merge.loc[df_merge['angle'].between(40, 100, 'right'), 'bin_angle'] = '100'
df_merge.loc[df_merge['angle'].between(100, 180, 'right'), 'bin_angle'] = '180'
df_merge['bin_angle'] = df_merge['bin_angle'].astype(float)

df_merge.loc[df_merge['angle_distant'].between(0, 40, 'both'), 'bin_watch'] = '40'
df_merge.loc[df_merge['angle_distant'].between(40, 100, 'right'), 'bin_watch'] = '100'
df_merge.loc[df_merge['angle_distant'].between(100, 180, 'right'), 'bin_watch'] = '180'
df_merge['bin_watch'] = df_merge['bin_watch'].astype(float)

df_merge.loc[df_merge['angle_in_proximity'].between(0, 40, 'both'), 'bin_prox_angle'] = '40'
df_merge.loc[df_merge['angle_in_proximity'].between(40, 100, 'right'), 'bin_prox_angle'] = '100'
df_merge.loc[df_merge['angle_in_proximity'].between(100, 180, 'right'), 'bin_prox_angle'] = '180'
df_merge['bin_prox_angle'] = df_merge['bin_prox_angle'].astype(float)

df_merge

In [ ]:
df_merge.to_csv('/Users/File Path/df_merge.csv')

In [ ]:
#Linear Mixed Effects Model for Angle DAta

In [ ]:
data = pd.read_csv('/Users/File Path/test.csv')

In [ ]:
md = smf.mixedlm("zscore ~ condition * surgery * bin_angle", data, groups=data["mouse"] ) 

mdf = md.fit()
print(mdf.summary())

In [ ]:
#Linear Mixed Effects Model for Behavioral Data

In [ ]:
data = pd.read_csv('/Users/File Path/NN Sniff Total.csv')

In [ ]:
baseline = list( data['baseline'] )
initial = list( data['initial'] )
extended = list( data['extended'] )

auc = baseline + initial + extended 

period = ['baseline']*len(baseline) + ['initial']*len(initial) + ['extended']*len(extended)
animal_info = data[['mouse', 'condition', 'surgery']]
animal_info_long = pd.concat([animal_info, animal_info, animal_info], axis=0)

animal_info_long['auc'] = auc
animal_info_long['period'] = period

animal_info_long.head()
df = animal_info_long.copy()
df.head()

In [ ]:
md = smf.mixedlm("auc ~ surgery * condition * period", df, groups=df["mouse"] ) 
# report python package, version, and this line in the manuscript. 

# md = smf.mixedlm("auc ~ surgery * condition * period", df, groups=df["mouse"], re_formula='1') #random effects formula
mdf = md.fit()
print(mdf.summary())

In [ ]:
sum = mdf.summary()
sum